In [1]:
!pip install textstat

    100% |████████████████████████████████| 3.0MB 14.1MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ec2-user/nltk_data...


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import boto3
import pandas as pd
import os.path
from os import path

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

def download(bucket, data_key):
    if path.exists(data_key):
        print('File '+data_key+' existed.')
        return False
    else:
        s3.Object(bucket, data_key).download_file(data_key)
        print('File '+data_key+' downloaded.')
        return True

In [4]:
bucket = 'wp-dw-sandbox'
url_prefix = 'yuy/tmp_yuy_30_mins_comments_sentiment/'
for obj in s3_client.list_objects(Bucket = bucket, Prefix = url_prefix)['Contents']:
#     filename, file_extension = os.path.splitext(obj['Key'])
    data_key = obj['Key']
    download(bucket, data_key)

File yuy/tmp_yuy_30_mins_comments_sentiment/000000_0 existed.
File yuy/tmp_yuy_30_mins_comments_sentiment/000001_0 existed.
File yuy/tmp_yuy_30_mins_comments_sentiment/000002_0 existed.
File yuy/tmp_yuy_30_mins_comments_sentiment/000003_0 existed.
File yuy/tmp_yuy_30_mins_comments_sentiment/000004_0 existed.
File yuy/tmp_yuy_30_mins_comments_sentiment/000005_0 existed.
File yuy/tmp_yuy_30_mins_comments_sentiment/000006_0 existed.
File yuy/tmp_yuy_30_mins_comments_sentiment/000007_0 existed.
File yuy/tmp_yuy_30_mins_comments_sentiment/000008_0 existed.
File yuy/tmp_yuy_30_mins_comments_sentiment/000009_0 existed.
File yuy/tmp_yuy_30_mins_comments_sentiment/000010_0 existed.
File yuy/tmp_yuy_30_mins_comments_sentiment/000011_0 existed.
File yuy/tmp_yuy_30_mins_comments_sentiment/000012_0 existed.
File yuy/tmp_yuy_30_mins_comments_sentiment/000013_0 existed.
File yuy/tmp_yuy_30_mins_comments_sentiment/000014_0 existed.
File yuy/tmp_yuy_30_mins_comments_sentiment/000015_0 existed.
File yuy

In [5]:
import dask.dataframe as dd
names = ['id', 'body']
dtype = {'id': str, 'body': str}
df = dd.read_csv(url_prefix+'*', sep='\01', header=None, names=names, dtype=dtype)
df = df.compute()

In [6]:
# import math
# def get_polarity_scores(row):
#     return sid.polarity_scores(str(row['body']))

def get_text(row):
    return str(row['body'])

import textstat

df['readability_sentence_count'] = df.apply(lambda row: textstat.sentence_count(get_text(row)), axis=1)
print('OK')
df['readability_lexicon_count'] = df.apply(lambda row: textstat.lexicon_count(get_text(row), removepunct=True), axis=1)
print('OK')
df['readability_char_count'] = df.apply(lambda row: textstat.char_count(get_text(row), ignore_spaces=True), axis=1)
print('OK')


OK
OK
OK


In [ ]:
from pandas.io.json import json_normalize

def only_dict(d):
    if isinstance(d, dict):
        return d
    else:
        return dict()

polarity_scores = json_normalize(df['polarity_scores'].apply(only_dict).tolist()).add_prefix('polarity_scores_')

In [44]:
df = df.join(polarity_scores)

In [8]:
df =  df.drop(['body'], axis=1)

In [52]:
df = df.reset_index(drop=True)

In [54]:
polarity_scores = json_normalize(df['polarity_scores'].apply(only_dict).tolist()).add_prefix('polarity_scores_')


In [55]:
polarity_scores

,polarity_scores_compound,polarity_scores_neg,polarity_scores_neu,polarity_scores_pos
0,-0.6233,0.124,0.747,0.129
1,0.9018,0.000,0.706,0.294
2,0.0000,0.000,1.000,0.000
3,0.5423,0.000,0.000,1.000
4,0.8271,0.000,0.573,0.427
5,0.8361,0.000,0.824,0.176
6,-0.6808,0.247,0.753,0.000
7,-0.5362,0.129,0.823,0.048
8,0.0000,0.000,1.000,0.000
9,0.6767,0.000,0.754,0.246


In [57]:
df = df.join(polarity_scores)

In [60]:
df =  df.drop(['polarity_scores'], axis=1)

In [10]:
df.to_csv('yuy/with_polarity_readability.tsv', sep='\t', index=False)

In [11]:
s3.Object(bucket, 'yuy/with_polarity_readability.tsv').upload_file('yuy/with_polarity_readability.tsv')